Ce notebook permet de simuler la donnée d'entrée du pipeline avec deux urls, de mettre à jour la configuration pour la Google Cloud Fonction, de déployer la Google Cloud Fonction et de la lancer lorsque cela a été déjà fait en ligne de commande.

# Etape 1: Simulation d'un fichier d'entrée 

In [1]:
import pandas as pd
import json

df = pd.DataFrame.from_dict({"url": ["https://www.groupedubreuil.com/","https://www.groupedubreuil.com/le-mot-des-presidents"]
                             , "id": ["0","1"]})

In [2]:
df

,id,url
0,0,https://www.groupedubreuil.com/
1,1,https://www.groupedubreuil.com/le-mot-des-pres...


# Etape 2: Création du fichier Json pour la GC Function

In [3]:
json_urls = json.load(open("/root/multisite-lighthouse-gcp/config.json"))

In [4]:
json_urls['source'] = df.to_dict('records')
json_urls['projectId'] = "lighthousegb"
json_urls['datasetId'] = "lighthouse"
json_urls['gcs']['bucketName'] = "gb-lighthousereports"

In [5]:
json_urls

{u'datasetId': 'lighthouse',
 u'gcs': {u'bucketName': 'gb-lighthousereports'},
 u'lighthouseFlags': {u'emulatedFormFactor': u'desktop',
  u'output': [u'html', u'csv']},
 u'minTimeBetweenTriggers': 300000,
 u'projectId': 'lighthousegb',
 u'pubsubTopicId': u'launch-lighthouse',
 u'source': [{'id': '0', 'url': 'https://www.groupedubreuil.com/'},
  {'id': '1', 'url': 'https://www.groupedubreuil.com/le-mot-des-presidents'}]}

In [6]:
with open("/root/multisite-lighthouse-gcp/config.json", 'w') as f:
    json.dump(json_urls, f, ensure_ascii=False)

# Etape 3: Déploiement de la Google Cloud Fonction

Cette partie peut prendre quelques minutes

In [7]:
import os

os.system("gcloud functions deploy launchLighthouse --trigger-topic launch-lighthouse --memory 2048 --timeout 540 --runtime=nodejs8")

0

# Etape 4: Utilisation de Google Cloud Fonction
Pour une première initialisation, il est nécessaire de le faire directement ligne de commande sans passer par Python.

In [10]:
os.system("gcloud pubsub topics publish launch-lighthouse --message all")

0